# 1. Import libraries and perform consistency checks from the reading

In [ ]:
import pandas as pd
import numpy as np
import os

In [2]:
path = r'/Users/danielmccormick/Desktop/CareerFoundry/Python/08-23 Instacart Basket Analysis'

In [3]:
df_ords = pd.read_csv(os.path.join(path, 'IC_Data', 'IC_prepared_data', 'orders_wrangled.csv'), index_col = [0])

In [4]:
df_prods = pd.read_csv(os.path.join(path, 'IC_Data', 'IC_original_data', 'products.csv'), index_col = False)

In [5]:
df_ords.head()

,order_id,user_id,user_order_count,orders_day_of_week,order_hour_of_day,days_since_prior_order
0,2539329,1,1,2,8,NaN
1,2398795,1,2,3,7,15.0
2,473747,1,3,3,12,21.0
3,2254736,1,4,4,7,29.0
4,431534,1,5,4,15,28.0


In [6]:
df_ords.describe()

,order_id,user_id,user_order_count,orders_day_of_week,order_hour_of_day,days_since_prior_order
count,3.421083e+06,3.421083e+06,3.421083e+06,3.421083e+06,3.421083e+06,3.214874e+06
mean,1.710542e+06,1.029782e+05,1.715486e+01,2.776219e+00,1.345202e+01,1.111484e+01
std,9.875817e+05,5.953372e+04,1.773316e+01,2.046829e+00,4.226088e+00,9.206737e+00
min,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,8.552715e+05,5.139400e+04,5.000000e+00,1.000000e+00,1.000000e+01,4.000000e+00
50%,1.710542e+06,1.026890e+05,1.100000e+01,3.000000e+00,1.300000e+01,7.000000e+00
75%,2.565812e+06,1.543850e+05,2.300000e+01,5.000000e+00,1.600000e+01,1.500000e+01
max,3.421083e+06,2.062090e+05,1.000000e+02,6.000000e+00,2.300000e+01,3.000000e+01


In [7]:
# performing a describe on the product dataframe but using a format found online so the results are not in scientific
# notation
df_prods.describe().apply(lambda x: x.apply('{0:.5f}'.format))

,product_id,aisle_id,department_id,prices
count,49693.00000,49693.00000,49693.00000,49693.00000
mean,24844.34514,67.77025,11.72843,9.99414
std,14343.71740,38.31677,5.85028,453.51969
min,1.00000,1.00000,1.00000,1.00000
25%,12423.00000,35.00000,7.00000,4.10000
50%,24845.00000,69.00000,13.00000,7.10000
75%,37265.00000,100.00000,17.00000,11.20000
max,49688.00000,134.00000,21.00000,99999.00000


In [ ]:
# There are two things that jump out when looking at the describe function for df_prods
# The first is that for product_id the count of rows is 46693, while the max id number is
# 46988. That would imply that there are five rows with the same product ID, which should
# not occur since it is the unique key value for each row in the data frame. 
# the second thing that jumps out is the maximum value for the price column
# based on the mean, 25%, 50%, and 75%, it seems unreasonable that there would be anything
# with a price of $99,999.00. This seems to imply there is an extreme outlier that 
# is most likely incorrect. That value could skew any pricing analysis we perform 

In [9]:
# creating mixed type column in a test dataframe 
df_test = pd.DataFrame()

In [10]:
df_test['mix'] = ['a', 'b', 1, True]

In [11]:
df_test.head()

,mix
0,a
1,b
2,1
3,True


# Function to test if there are any mixed type columns in a data frame 

In [12]:
for col in df_test.columns.tolist():
  weird = (df_test[[col]].applymap(type) != df_test[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (df_test[weird]) > 0:
    print (col)

mix


In [13]:
# updating test data frame to be all type string 
df_test = df_test['mix'].astype('str')

In [14]:
# using a sum to count how many null values there are in each column of df_prods 
df_prods.isnull().sum()

product_id        0
product_name     16
aisle_id          0
department_id     0
prices            0
dtype: int64

In [15]:
# creating a data frame that is just the rows with nulls in the product_name column 
df_nan = df_prods[df_prods['product_name'].isnull()==True]

In [16]:
df_nan

,product_id,product_name,aisle_id,department_id,prices
33,34,NaN,121,14,12.2
68,69,NaN,26,7,11.8
115,116,NaN,93,3,10.8
261,262,NaN,110,13,12.1
525,525,NaN,109,11,1.2
1511,1511,NaN,84,16,14.3
1780,1780,NaN,126,11,12.3
2240,2240,NaN,52,1,14.2
2586,2586,NaN,104,13,12.4
3159,3159,NaN,126,11,13.1


In [17]:
# checking how many rows are in df_prods 
df_prods.shape

(49693, 5)

In [18]:
# creating a new clean data frame that excludes all rows with null values in the product_name column 
df_prods_clean = df_prods[df_prods['product_name'].isnull() == False]

In [19]:
# confirming that the new clean data frame is 16 rows smaller than the original to confirm the correct number
# of rows were removed 
df_prods_clean.shape

(49677, 5)

In [20]:
# returns True if the row is a duplicate 
df_prods_clean.duplicated()

0        False
1        False
2        False
3        False
4        False
         ...  
49688    False
49689    False
49690    False
49691    False
49692    False
Length: 49677, dtype: bool

In [21]:
# creating a dataframe of only the duplicated rows from df_ords
df_dups = df_prods_clean[df_prods_clean.duplicated()]

In [22]:
df_dups

,product_id,product_name,aisle_id,department_id,prices
462,462,Fiber 4g Gummy Dietary Supplement,70,11,4.8
18459,18458,Ranger IPA,27,5,9.2
26810,26808,Black House Coffee Roasty Stout Beer,27,5,13.4
35309,35306,Gluten Free Organic Peanut Butter & Chocolate ...,121,14,6.8
35495,35491,Adore Forever Body Wash,127,11,9.9


In [23]:
df_prods_clean.shape

(49677, 5)

In [24]:
# creating a data frame that drops all duplicated rows and then confirming that it is five rows shorter than the
# original since we confirmed above there are five duplicated rows in the dataframe 
df_prods_clean_no_dups=df_prods_clean.drop_duplicates()

In [25]:
df_prods_clean_no_dups.shape

(49672, 5)

# 2. running describe on the order data frame 

In [26]:
df_ords.describe()

,order_id,user_id,user_order_count,orders_day_of_week,order_hour_of_day,days_since_prior_order
count,3.421083e+06,3.421083e+06,3.421083e+06,3.421083e+06,3.421083e+06,3.214874e+06
mean,1.710542e+06,1.029782e+05,1.715486e+01,2.776219e+00,1.345202e+01,1.111484e+01
std,9.875817e+05,5.953372e+04,1.773316e+01,2.046829e+00,4.226088e+00,9.206737e+00
min,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,8.552715e+05,5.139400e+04,5.000000e+00,1.000000e+00,1.000000e+01,4.000000e+00
50%,1.710542e+06,1.026890e+05,1.100000e+01,3.000000e+00,1.300000e+01,7.000000e+00
75%,2.565812e+06,1.543850e+05,2.300000e+01,5.000000e+00,1.600000e+01,1.500000e+01
max,3.421083e+06,2.062090e+05,1.000000e+02,6.000000e+00,2.300000e+01,3.000000e+01


In [27]:
# 2. found a method online for getting the results not in scientific notation 
df_ords.describe().apply(lambda x: x.apply('{0:.5f}'.format))

,order_id,user_id,user_order_count,orders_day_of_week,order_hour_of_day,days_since_prior_order
count,3421083.00000,3421083.00000,3421083.00000,3421083.00000,3421083.00000,3214874.00000
mean,1710542.00000,102978.20806,17.15486,2.77622,13.45202,11.11484
std,987581.73982,59533.71779,17.73316,2.04683,4.22609,9.20674
min,1.00000,1.00000,1.00000,0.00000,0.00000,0.00000
25%,855271.50000,51394.00000,5.00000,1.00000,10.00000,4.00000
50%,1710542.00000,102689.00000,11.00000,3.00000,13.00000,7.00000
75%,2565812.50000,154385.00000,23.00000,5.00000,16.00000,15.00000
max,3421083.00000,206209.00000,100.00000,6.00000,23.00000,30.00000


In [28]:
# 2. Based on the results of the describe function above it does not seem as thought there are any obvious issues
# with the data set. The minimum for order_id, user_id, and order_number is 1 because there would not be an id of 0
# and no one could have an order number zero since the first order they would make would be order 1. For these same
# three columns the maximum all seem reasonable as they just measure the number of order and users, and based on 
# order_number the most orders by any one user is 100, which seems possible. For order_dow, order_hour_of_day 
# and days_since_prior_order the minimum is 0 which makes sense because day of the week this would be the first day 
# of the week, for hours of the day it would be the first hour, and for days since prior order it means they ordered
# twice within the same day which is possible. Order dow of the week max of 6 makes sense because that would be the 
# seventh day of the week, hour of day max of 23 makes sense cause that would be the 24th hour of the day, and days 
# since last order max of 30 seems reasonable because that would mean the longest stretch between orders for a user
# was about one month. 
# one thing that does seems strange is the count for days_since_prior_order is 3,214,874, while for all other columns
# it is 3,421,083. This would seem to suggest there are some null values in that column in certain rows of data. 

# 3. Function to see if there are any mixed type columns in df_ords.

In [29]:
# It does not return any results, so 
# there are no mixed type columns in the data frame 
for col in df_ords.columns.tolist():
  weird = (df_ords[[col]].applymap(type) != df_ords[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (df_ords[weird]) > 0:
    print (col)

# 4. There are no mixed type data columns in the data frame

# 5. Running a check for missing values in orders dataset

In [31]:
# There are 206209 null values in the days_since_prior_order column of the df_ords dataframe
# This is because there is a null value in that field for the first row associated with each user_id
# if it a user's first order then there would not be any days since last order.
df_ords.isnull().sum()

order_id                       0
user_id                        0
user_order_count               0
orders_day_of_week             0
order_hour_of_day              0
days_since_prior_order    206209
dtype: int64

# 6. Dealing with the missing values in the order data set

In [72]:
# Since the values are missing because it is the user's first order and there cannot be any days since prior order for 
# a user's first order, I will leave the NaN values in the dataframe. 

In [32]:
df_ords_clean = df_ords

In [33]:
df_ords_clean.isnull().sum()

order_id                       0
user_id                        0
user_order_count               0
orders_day_of_week             0
order_hour_of_day              0
days_since_prior_order    206209
dtype: int64

# 7. Running a check for any duplicate values in the orders data

In [34]:
df_ords_clean.duplicated()

0          False
1          False
2          False
3          False
4          False
           ...  
3421078    False
3421079    False
3421080    False
3421081    False
3421082    False
Length: 3421083, dtype: bool

In [35]:
df_ords_dups = df_ords_clean[df_ords_clean.duplicated()]

In [36]:
df_ords_dups

,order_id,user_id,user_order_count,orders_day_of_week,order_hour_of_day,days_since_prior_order


# 8. Based on df_ords_dups not having any rows of data there are not any duplicate rows in the df_ords_clean data set 

# 9. Exporting the clean data frames 

In [38]:
df_prods_clean_no_dups.to_csv(os.path.join(path, 'IC_Data', 'IC_prepared_data', 'products_clean.csv'))

In [39]:
df_ords_clean.to_csv(os.path.join(path, 'IC_Data', 'IC_prepared_data', 'orders_clean.csv'))